In [1]:
!conda install -q -y xarray scipy dask netCDF4

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - dask
    - netcdf4
    - scipy
    - xarray


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bokeh-2.3.3                |   py37h89c1867_0         8.2 MB  conda-forge
    cftime-1.5.0               |   py37h6f94858_0         224 KB  conda-forge
    curl-7.78.0                |       hea6ffbf_0         148 KB  conda-forge
    cytoolz-0.11.0             |   py37h5e8e339_3         403 KB  conda-forge
    dask-2021.7.1              |     pyhd8ed1ab_0           4 KB  conda-forge
    dask-core-2021.7.1         |     pyhd8ed1ab_0         750 KB  conda-forge
    distributed-2021.7.1       |   py37h89c1867_0         1.0 MB  conda-forge
    hdf4-4.2.15                |       h10796ff_3         950 KB  conda-forge
    heapdict-1.0.1             |             py_0         

In [2]:
!conda install -q -y -c conda-forge esmpy

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - esmpy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    esmf-8.1.1                 | nompi_hb4f3075_0        27.3 MB  conda-forge
    esmpy-8.1.1                |nompi_py37hff16fce_0         173 KB  conda-forge
    netcdf-fortran-4.5.3       |nompi_hf3f1587_104         1.3 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        28.7 MB

The following NEW packages will be INSTALLED:

  esmf               conda-forge/linux-64::esmf-8.1.1-nompi_hb4f3075_0
  esmpy              conda-forge/linux-64::esmpy-8.1.1-nompi_py37hff16fce_0
  netcdf-fortran     conda-forge/linux-64::netcdf-fortran-4.5.3-nompi_hf3f1587_104


Preparing transaction: ...working... done
Verifying transaction: ...work